# Deep Learning methods: BERT model 

## Jose Antonio Jijon Vorbeck

In this notebook, we will apply a BERT model to the tweets data set to perform a sentiment analysis. 

We will compare the results obtained from this model to those obtained using classical ML models.

[BERT - Kaggle](https://www.kaggle.com/nayansakhiya/text-classification-using-bert)